Adapted from the notebook found at [How to Build a Law Bot](https://lawyerist.com/how-build-law-bot/)

## Install libraries

If you haven't already, you may need to install some dependencies. On the command line, run the following to install/update gspread, oauth2client, PyOpenSSL, and python-twitter.
```
pip install gspread
pip install --upgrade oauth2client
pip install PyOpenSSL
pip install python-twitter
```
Library installs are one and done. So after doing this once, you should be all set. 

## Import modules and set variables

Now we're getting into the bot's code. This is what will run every time your bot is called. 

You will need to create a new Google Sheet (same instructions as [last time](https://lawyerist.com/126074/online-forms-meet-local-document-automation-cut-and-paste-coding/)). Delete rows 2-999. This is because the code below appends values to the end of your sheet. So if you fail to remove rows 2-999, values will be appended to row 1000. Additionally, it looks at the last row of the sheet for your old values. So right off the bat it will be looking at your one solitary row. Also, delete columns D through Z to avoind having to print a bunch of empty columns.

As for a Twitter account and Twitter credentials, follow the instruction in [this post](https://lawyerist.com/?p=127093). 

In [32]:
# Load the module for visiting and reading websites.
import urllib.request
# Load the module for running regular expressions (regex).
import re 
# Load the module for date and time stuff.
import datetime
# Define the variable now as equal to the current date and time.
now = datetime.datetime.now()

In [43]:
# Set the URL you want to scrape.
#url_1 = "https://www.bostoninjurylawyerblog.com/category/medical-malpractice"
url_1 ="https://www.bostonpersonalinjuryattorneyblog.com/category/medical-malpractice/feed"
url_2 = "https://www.bostoninjurylawyerblog.com/category/medical-malpractice/feed"

# If you want to scrape data from multiple pages, you can, 
# just replicate the above and below but change url_1 to url_2 et al.

In [34]:
# Load the module for accessing Google Sheets.
import gspread
# Load the module needed for securely communicating with Google Sheets.
from oauth2client.service_account import ServiceAccountCredentials
# The scope for your access credentials
scope = ['https://spreadsheets.google.com/feeds']

# Your spreadsheet's ID
document_key = "1-sYU2ur4kpERjl0dFr_ZcpsVRMWG_J1n98r9ExDOClE" 
#              ^^^^^^^^^^^ SWAP OUT FOR YOUR DOCUMENT ID/KEY
# Your Google project's .json key
credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../../../CodingProject2-517c5ae88a4a.json', scope)
#                                                                              ^^^^^^^^ SWAP OUT FOR YOUR JSON KEY
# Use your credentials to authorize yourself.
gc = gspread.authorize(credentials)
# Open up the Sheet with the defined ID.
wks = gc.open_by_key(document_key)

#########################################
#
#  NOTE: The name of the sheet you are 
#  trying to access should be in the 
#  parenthetical below (e.g., Data). By
#  Default this is probably "Sheet1".
#
#########################################
worksheet = wks.worksheet("Sheet1")

# Count the number of rows in your Sheet &
# resize to remove blank rows.
worksheet.resize(worksheet.row_count)

In [35]:
# Print out the old values stored in your sheet 
# Note: The first time you run this code, it will be empty as nothing has yet to be stored in your sheet.

print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

['0', '0', '0', '0']


In [36]:
# Import the relevant Twitter libraries so you can use Twitter.
import twitter
from twitter import TwitterError

# create the following four text files and add them to the same diretctry as you 
# Google API key. In each file add the appropriate value found when retrieving your 
# Twitter credentials

with open('../../../../../key.txt', 'r') as myfile:
    key=myfile.read()
    
with open('../../../../../secret.txt', 'r') as myfile:
    secret=myfile.read()
    
with open('../../../../../token_key.txt', 'r') as myfile:
    token_key=myfile.read()

with open('../../../../../token_secret.txt', 'r') as myfile:
    token_secret=myfile.read()

# Set you Twitter API credentials.
api = twitter.Api(consumer_key=key,
                  consumer_secret=secret,
                  access_token_key=token_key,
                  access_token_secret=token_secret)

## Read the contents of your first webpage

When you run the next cell, your program will visit the first URL you defined above. It will then print out that page's HTML. 

In [44]:
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
print(p_1)

b'<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"\n\txmlns:content="http://purl.org/rss/1.0/modules/content/"\n\txmlns:wfw="http://wellformedweb.org/CommentAPI/"\n\txmlns:dc="http://purl.org/dc/elements/1.1/"\n\txmlns:atom="http://www.w3.org/2005/Atom"\n\txmlns:sy="http://purl.org/rss/1.0/modules/syndication/"\n\txmlns:slash="http://purl.org/rss/1.0/modules/slash/"\n\t>\n\n<channel>\n\t<title>medical malpractice &#8211; Boston Personal Injury Attorney Blog</title>\n\t<atom:link href="https://www.bostonpersonalinjuryattorneyblog.com/category/medical-malpractice/feed" rel="self" type="application/rss+xml" />\n\t<link>https://www.bostonpersonalinjuryattorneyblog.com</link>\n\t<description>Published by Boston, Massachusetts Personal Injury Lawyer \xe2\x80\x94 Jeffrey S. Glassman</description>\n\t<lastBuildDate>Tue, 10 Oct 2017 16:45:06 +0000</lastBuildDate>\n\t<language>en-US</language>\n\t<sy:updatePeriod>hourly</sy:updatePeriod>\n\t<sy:updateFrequency>1</sy:updateFrequency>\n\t\

In [42]:
p_2 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_2).read()
print(p_2)

b'<!DOCTYPE html>\n<html class="no-js displaymode-full" itemscope itemtype="http://schema.org/Blog" lang="en-US">\n\t<head prefix="og: http://ogp.me/ns#" >\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\t\t<title>medical malpractice Category Archives &mdash; Boston Personal Injury Attorney Blog Published by Boston, Massachusetts Personal Injury Lawyer \xe2\x80\x94 Jeffrey S. Glassman</title>\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0" />\n\n\t\t<!--[if lt IE 9]>\n\t\t\t<script src="//html5shiv.googlecode.com/svn/trunk/html5.js"></script>\n\t\t<![endif]-->\n\n\n\n\n\t\t<link rel="stylesheet" type="text/css" media="all" href="/jresources/v/20170921003617/css/main.css" />\n\n\n\t\t<script type="text/javascript">\n\t\t\tdocument.documentElement.className = document.documentElement.className.replace(/\\bno-js\\b/,"js");\n\t\t</script>\n\n\t\t<meta name="referrer" content="origin" />\n\t\t<meta name="referrer" content="unsafe-url" 

------------------------------------

# One Data Point, One Match

## Parse the site's contents 

Scan the above HTML for the content you are trying to extract. Cut and paste the HTML above into the TEST STRING box over at [Regex 101](https://regex101.com/) and craft a regex that captures your desired content. Be sure to use the Python flavor.

Remember the parenthetical is the group you're pulling out. Once you have a working regex, plug it into the code below, and run the cell. If it worked, you'll see you scraped data as an output. 

In [47]:
res_1 = re.search(b"item>\s*<title>(.*)</title>\s*<link>(.*)</link>",p_1)
output_1 = res_1.group(1).decode('UTF-8')
output_2 = res_1.group(2).decode('UTF-8')
print(output_1)
print(output_2)

Medical Malpractice Cases and Medical Battery
https://www.bostonpersonalinjuryattorneyblog.com/2017/08/medical-malpractice-cases-medical-battery.html


In [48]:
res_2 = re.search(b"item>\s*<title>(.*)</title>\s*<link>(.*)</link>",p_2)
output_3 = res_2.group(1).decode('UTF-8')
output_4 = res_2.group(2).decode('UTF-8')
print(output_3)
print(output_4)

AttributeError: 'NoneType' object has no attribute 'group'

## Post to Twitter and Save to Google (Two Data Point, One Match)

In [39]:
if ((res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1
          and (worksheet.row_values(worksheet.row_count)[2]) != output_2)
    or
    (res_2 and (worksheet.row_values(worksheet.row_count)[3]) != output_3
          and (worksheet.row_values(worksheet.row_count)[4]) != output_4)):
    # same as above but now comparing two values
    
    if (res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1
          and (worksheet.row_values(worksheet.row_count)[2]) != output_2):

        try:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_1,output_2))
            print(status.text)
        except TwitterError:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_1,output_2))
            print(status.text)        

    if (res_2 and (worksheet.row_values(worksheet.row_count)[3]) != output_3
          and (worksheet.row_values(worksheet.row_count)[4]) != output_4):

        try:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_3,output_4))
            print(status.text)
        except TwitterError:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_3,output_4))
            print(status.text)        

    # Save to Google only after Tweeting
    worksheet.append_row([now,output_1,output_2,output_3,output_4])    

Jury Selection in Personal Injury Cases in Boston https://t.co/no95bRXEkv


NameError: name 'res_2' is not defined

In [ ]:
print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################